In [1]:
!pip install -q -r requirements.txt

# Bölüm 1: Genel Sayfa Tıklama İşlemleri

In [2]:
import sys
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import re
from fake_useragent import UserAgent



In [3]:
ua = UserAgent()
ua.random   

url = "https://tez.yok.gov.tr/UlusalTezMerkezi/"


In [4]:
import undetected_chromedriver as uc
def get_driver(url):
    ua = UserAgent()
    ua.random       
    options = uc.ChromeOptions()
    options.add_argument(f'user-agent={ua.random}')
    driver = uc.Chrome(options=options)
    driver.get(url)
    time.sleep(3)
    return driver



In [ ]:
# # # choose pop up
# import time
# driver = get_driver(url)
# time.sleep(10)
# driver.switch_to.window(driver.window_handles[1])

# soup = BeautifulSoup(driver.page_source, 'html.parser')


# üniversiteler = []
# for i in soup.find_all("a"):
#     üniversiteler.append(i.text)
# üniversiteler = üniversiteler[1:]
# len(üniversiteler)
# pd.Series(üniversiteler, name="üniversiteler").to_csv("üniversiteler-15-03-2025.csv", index=False)
# # üniversiteler = pd.read_csv("üniversiteler.csv").üniversiteler.to_list()

In [7]:
def click_tarama(driver):
    tarama_path = '//*[@id="navigation2"]/ul/li[2]/a'
    driver.find_element(By.XPATH, tarama_path).click()
    time.sleep(3)    

In [8]:
def üniversite_seç(driver, üniversite):
    path = '//*[@id="tabs-1"]/form/table/tbody/tr/td/table/tbody/tr[2]/td[2]/input[1]'
    driver.find_element(By.XPATH, path).click()
    driver.switch_to.window(driver.window_handles[1])
    # click by text
    driver.find_element(By.LINK_TEXT, üniversite).click()
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[0])

    

In [9]:
def click_tez_türü(driver, tür="Tıpta Uzmanlık"):
    tez_türü_path = '//*[@id="tabs-1"]/form/table/tbody/tr/td/table/tbody/tr[2]/td[4]/select'
    driver.find_element(By.XPATH, tez_türü_path).click()
    # choose dropdown menu by text
    time.sleep(1)
    driver.find_element(By.XPATH, tez_türü_path).send_keys(tür)   
    time.sleep(1) 



In [10]:
def click_bul(driver):
    bul_path = '//*[@id="tabs-1"]/form/table/tbody/tr/td/table/tbody/tr[8]/td/input[3]'
    driver.find_element(By.XPATH, bul_path).click()
    time.sleep(2)

In [11]:
def sonuç_sayısı(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    sonuç_sayısı = soup.find("div", {"id": "divuyari"}).text
    sonuç_sayısı = re.findall(r'\d+', sonuç_sayısı)
    return int(sonuç_sayısı[0])

In [12]:
driver = get_driver(url)

In [13]:
click_tarama(driver)

In [14]:
üniversite_seç(driver, üniversiteler[0])

In [15]:
click_tez_türü(driver)

In [16]:
click_bul(driver)

In [17]:
sonuç_sayısı(driver)

275

In [18]:
def yıl_aralığı(driver, yıl1, yıl2):
    # yıl2 yıl1 den büyük olmalı
    yıl1_path = '//*[@id="tabs-1"]/form/table/tbody/tr/td/table/tbody/tr[2]/td[6]/select[1]'
    yıl2_path = '//*[@id="tabs-1"]/form/table/tbody/tr/td/table/tbody/tr[2]/td[6]/select[2]'
    driver.find_element(By.XPATH, yıl1_path).send_keys(yıl1)
    driver.find_element(By.XPATH, yıl2_path).send_keys(yıl2)
    time.sleep(1)



# Bölüm 2: Sonuçları çekme

In [19]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
# text/javascript
data = soup.find_all("script", {"type": "text/javascript"})[5]
data = str(data)

In [20]:
# regex ile verileri parse etme
# tez no: >484903</span>" arasında kalan sayılar
tez_no = re.findall(r'(?<=>)\d+(?=</span>)', data)
# yıl-->  '''age: "2018",''' arasında kalan sayılar
yıl = re.findall(r'(?<=age: ")\d+(?=",)', data)
# title--> '''weight: "Tiroidin benign ve malign neoplazmlarının </span>",''' arasında kalan text
title = re.findall(r'(?<=weight: ")(.*?)(?=</span>)', data)
# üniversite--> '''uni:"Abant İzzet Baysal Üniversitesi",''' arasında kalan text
üniversite = re.findall(r'(?<=uni:")(.*?)(?=",)', data)
# tez_türü --> '''important: "Tıpta Uzmanlık",''' arasında kalan text
tez_türü = re.findall(r'(?<=important: ")(.*?)(?=",)', data)
# konu --> '''someDate: "Patoloji = Pathology",''' arasında kalan text
konu = re.findall(r'(?<=someDate: ")(.*?)(?=",)', data)
# tez_detay--> '''#FF0000;\" text text > arasında kalan text
tez_detay  = r'#FF0000;\\\"(.+?)>'
tez_detay = re.findall(tez_detay, data)


In [21]:
def get_table(driver):
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # text/javascript
    data = soup.find_all("script", {"type": "text/javascript"})[5]
    data = str(data)    
    # tez no: >484903</span>" arasında kalan sayılar
    tez_no = re.findall(r'(?<=>)\d+(?=</span>)', data)
    yıl = re.findall(r'(?<=age: ")\d+(?=",)', data)
    title = re.findall(r'(?<=weight: ")(.*?)(?=</span>)', data)
    üniversite = re.findall(r'(?<=uni:")(.*?)(?=",)', data)
    tez_türü = re.findall(r'(?<=important: ")(.*?)(?=",)', data)
    konu = re.findall(r'(?<=someDate: ")(.*?)(?=",)', data)
    tez_detay  = r'#FF0000;\\\"(.+?)>'
    tez_detay = re.findall(tez_detay, data)
    
    return pd.DataFrame({"tez_no": tez_no,
                         "yıl": yıl,
                         "title": title,
                         "üniversite": üniversite,
                         "tez_türü": tez_türü,
                         "konu": konu,
                         "tez_detay": tez_detay})


In [24]:
df = get_table(driver)

In [2]:
def get_abstract(driver,tez_detay,tez_no):
    driver.execute_script(tez_detay)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tezno = soup.find_all("tr", {"class": "renkp"})[0].find_all("td")[0].text.replace("\n\t","").strip()
    while tezno != tez_no:
        driver.execute_script(tez_detay)
        time.sleep(0.2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tezno = soup.find_all("tr", {"class": "renkp"})[0].find_all("td")[0].text.replace("\n\t","").strip()
        time.sleep(0.1)
    try:
        en_absract = soup.find(id = 'td1').text
    except:
        en_absract = np.nan
    try:
        tr_absract = soup.find(id = 'td0').text
    except:
        tr_absract = np.nan
    return en_absract, tr_absract

In [3]:
get_abstract(driver,tez_detay[150], df.tez_no[150])

NameError: name 'driver' is not defined

In [38]:
tez_detay[:3]

[" onclick=tezDetay('p3M1GrttjeK-zqVNDXNCFg','LfHnFSa4-F9wIA8tZssCYQ')",
 " onclick=tezDetay('k_56RsiVKppBdWVEPqLv_w','1jXSo_rMnc1ifcmy2o1L3w')",
 " onclick=tezDetay('p_-c_oZf7xzNGwK2UHiQ4Q','KRspEfA5dL2jTelR3lPRGQ')"]

In [27]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
tezno = soup.find_all("tr", {"class": "renkp"})[0].find_all("td")[0].text.replace("\n\t","").strip()

In [28]:
tezno

'132528'